<a href="https://colab.research.google.com/github/vipashaaV321/User-Intent-Modeling/blob/main/weight_update_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

# Sample user profiles as dictionaries (movie is key, and importance weight is the value)
user_profiles = {
    'Star Wars (1977)': 5.0,
    'Shawshank Redemption, The (1994)': 4.5,
    'Terminator 2: Judgment Day (1991)': 4.0,
    # Add more movies and weights
}

# Sample movie list
movie_list = [
    'Matrix, The (1999)',
    'Jurassic Park (1993)',
    'Pulp Fiction (1994)',
    # Add more movies
]

# Define neural network model with attention mechanism
embedding_dim = 32

# Input layer for selected item
selected_item_input = tf.keras.layers.Input(shape=(1,), name="selected_item")

# Input layer for user profile
user_profile_input = tf.keras.layers.Input(shape=(len(movie_list),), name="user_profile")

# Embedding layer for items
item_embedding = tf.keras.layers.Embedding(input_dim=len(movie_list), output_dim=embedding_dim)(selected_item_input)

# Compute attention weights
attention_weights = tf.keras.layers.Dot(axes=1)([item_embedding, user_profile_input])
attention_weights = tf.keras.layers.Flatten()(attention_weights)
attention_weights = tf.keras.layers.Softmax()(attention_weights)

# Weighted sum to update user profile
weighted_sum = tf.keras.layers.Dot(axes=1)([attention_weights, item_embedding])

# Add the weighted sum to the user profile
updated_user_profile = tf.keras.layers.Add()([user_profile_input, weighted_sum])

# Create the model
model = tf.keras.Model(inputs=[selected_item_input, user_profile_input], outputs=updated_user_profile)

model.compile(optimizer='adam', loss='mean_squared_error')

# Simulate the conversation
while True:
    print("Movie List:")
    for i, movie in enumerate(movie_list):
        print(f"{i + 1}. {movie}")

    user_choice = int(input("Select a movie (1-3) or 0 to exit: "))

    if user_choice == 0:
        break

    if 1 <= user_choice <= len(movie_list):
        selected_movie = movie_list[user_choice - 1]
        print(f"Selected Movie: {selected_movie}")

        # Simulate user feedback (for example, user can provide a rating)
        user_rating = float(input(f"Rate {selected_movie} (0-5): "))

        # Update user profile using the neural network
        selected_item_index = movie_list.index(selected_movie)
        user_profile_array = np.array([user_profiles.get(movie, 0) for movie in movie_list])
        user_profile_array = user_profile_array.reshape(1, -1)

        updated_profile = model.predict([np.array([selected_item_index]), user_profile_array])
        user_profiles[selected_movie] = user_rating

        print(f"Updated User Profile: {user_profiles}")

print("Conversation ended.")


ValueError: ignored

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load MovieLens 100K dataset (You need to download it and adjust the file paths)
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Preprocess the dataset
# You may need to do additional preprocessing depending on your requirements
# In this example, we will use user ratings as implicit feedback
ratings['rating'] = 1

# Create user profiles as a dictionary with movie importance weights
user_profiles = {
    'Star Wars (1977)': 5.0,
    'Shawshank Redemption, The (1994)': 4.5,
    'Terminator 2: Judgment Day (1991)': 4.0,
    # Add more movies and weights
}

# Define user and movie indices
user_ids = np.arange(0, len(user_profiles))
movie_ids = np.arange(0, len(movies))

# Create training and testing data
user_train, user_test, movie_train, movie_test, ratings_train, ratings_test = train_test_split(
    user_ids, movie_ids, ratings, test_size=0.2, random_state=42)

# Build the recommendation model
embedding_dim = 32

user_input = tf.keras.layers.Input(shape=(1,), name="user_id")
movie_input = tf.keras.layers.Input(shape=(1,), name="movie_id")

user_embedding = tf.keras.layers.Embedding(input_dim=len(user_profiles), output_dim=embedding_dim)(user_input)
movie_embedding = tf.keras.layers.Embedding(input_dim=len(movies), output_dim=embedding_dim)(movie_input)

user_vecs = tf.keras.layers.Flatten()(user_embedding)
movie_vecs = tf.keras.layers.Flatten()(movie_embedding)

interaction = tf.keras.layers.Dot(axes=1)([user_vecs, movie_vecs])
model = tf.keras.Model(inputs=[user_input, movie_input], outputs=interaction)

model.compile(optimizer='adam', loss='mean_squared_error')

# Define a function to get movie recommendations
def get_movie_recommendations(user_profile, top_n=10):
    user_ratings = []
    movie_ids = np.arange(0, len(movies))
    user_id = np.zeros(len(movie_ids))

    for movie, rating in user_profile.items():
        movie_idx = movies[movies['title'] == movie].index[0]
        user_ratings.append(rating)
        user_id[movie_idx] = 1

    user_id = user_id.reshape(1, -1)
    user_ratings = np.array(user_ratings).reshape(1, -1)

    predicted_ratings = model.predict([user_id, movie_ids])
    predicted_ratings = predicted_ratings.reshape(-1)

    top_movie_indices = predicted_ratings.argsort()[-top_n:][::-1]

    top_movies = movies.iloc[top_movie_indices]
    return top_movies

# Simulate a conversation
while True:
    user_feedback = input("Enter feedback (Movie Name or 'End' to finish): ")

    if user_feedback.lower() == 'end':
        break

    if user_feedback in user_profiles:
        print(f"You've already rated {user_feedback}")
        continue

    user_rating = float(input(f"Rate {user_feedback} (0-5): "))
    user_profiles[user_feedback] = user_rating

    recommendations = get_movie_recommendations(user_profiles)
    print("Recommended Movies:")
    print(recommendations[['title', 'genres']])

print("Conversation ended.")

# You can add more features like updating user profiles and model training at each iteration.


FileNotFoundError: ignored

In [ ]:
import numpy as np
from surprise import Dataset, Reader, SVD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the MovieLens dataset using Surprise
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(1, 5))
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()

# Preprocess the dataset for text input
# For the sake of this example, let's assume we have some conversational data
conversations = {
    'hello': 'Welcome! How can I help you with movie recommendations?',
    'good movies': 'Sure! What genre are you interested in?'
    # Add more conversation pairs as needed
}

tokenizer = Tokenizer()
tokenizer.fit_on_texts(conversations.values())
vocab_size = len(tokenizer.word_index) + 1

# Define an LSTM-based conversational model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the conversation model
input_sequences = tokenizer.texts_to_sequences(conversations.values())
input_padded = pad_sequences(input_sequences, maxlen=1)
target_sequences = np.zeros((len(conversations), vocab_size))
for i, sequence in enumerate(input_sequences):
    target_sequences[i, sequence] = 1

model.fit(input_padded, target_sequences, epochs=50)

# User interaction loop
context = 'hello'  # Initial conversation prompt
user_input = ''
while user_input.lower() != 'exit':
    user_input = input(f'User: {context}\n')
    input_sequence = tokenizer.texts_to_sequences([user_input])
    input_padded = pad_sequences(input_sequence, maxlen=1)
    predicted_word = np.argmax(model.predict(input_padded), axis=-1)

    # Get the next context based on predicted word
    for word, index in tokenizer.word_index.items():
        if index == predicted_word:
            context = word
            break

    # Perform movie recommendation based on the context
    # (Here we'll use the Surprise library for recommendations)
    if context == 'good movies':
        recommendations = model.recommend_items_for_user(1, 10)
        print("Recommended Movies:")
        for movie_id, _ in recommendations:
            print(f"Movie ID: {movie_id}")

print('Exiting conversation.')


ModuleNotFoundError: ignored

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Simulated data (replace with your dataset)
data = pd.DataFrame({
    'user_id': [1, 2, 1, 3, 2, 4],
    'movie_id': [101, 102, 103, 104, 102, 105],
    'rating': [5, 4, 3, 5, 4, 2]
})

# Encode user and movie IDs
user_encoder = LabelEncoder()
data['user_id'] = user_encoder.fit_transform(data['user_id'])
movie_encoder = LabelEncoder()
data['movie_id'] = movie_encoder.fit_transform(data['movie_id'])

# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the model
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')

user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=64)(user_input)
movie_embedding = Embedding(input_dim=len(movie_encoder.classes_), output_dim=64)(movie_input)

attention = Attention()([user_embedding, movie_embedding])
combined = keras.layers.concatenate([user_embedding, movie_embedding, attention])

x = LSTM(64)(combined)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='linear')(x)

model = keras.Model(inputs=[user_input, movie_input], outputs=output)
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit([train_data['user_id'], train_data['movie_id']], train_data['rating'], epochs=10, batch_size=64)

# Evaluate the model on the test data
test_loss = model.evaluate([test_data['user_id'], test_data['movie_id']], test_data['rating'])
print(f'Test loss: {test_loss}')

# Make movie recommendations for a specific user
user_id = 0  # Replace with the user ID you want to make recommendations for
# user_movie_ids = data[data['user_id'] == user_id]['movie_id'].unique()
# user_movie_ids = user_movie_ids[np.random.choice(len(user_movie_ids), size=5, replace=False)]

# Sample a subset of movie_ids for recommendation
sample_size = min(5, len(user_movie_ids))  # Choose a sample size of 5 or less
user_movie_ids = user_movie_ids[np.random.choice(len(user_movie_ids), size=sample_size, replace=False)]

# Repeat user_movie_ids for all movies in the dataset
user_movie_ids = np.tile(user_movie_ids, len(movie_encoder.classes_))
user_ids = np.repeat(user_id, len(user_movie_ids))

predicted_ratings = model.predict([user_ids, user_movie_ids]).flatten()
movie_ids_sorted = np.argsort(predicted_ratings)[::-1]
recommended_movie_ids = movie_ids_sorted[:10]

recommended_movie_titles = movie_encoder.inverse_transform(recommended_movie_ids)

print(f'Recommended Movies for User {user_id}:')
for i, movie_title in enumerate(recommended_movie_titles):
    print(f'{i + 1}. {movie_title}')


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 13.4869
Epoch 2/10
1/1 [==============================] - 0s 13ms/step - loss: 13.4034
Epoch 3/10
1/1 [==============================] - 0s 11ms/step - loss: 13.3187
Epoch 4/10
1/1 [==============================] - 0s 13ms/step - loss: 13.2390
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 13.1604
Epoch 6/10
1/1 [==============================] - 0s 18ms/step - loss: 13.0748
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 12.9840
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 12.8888
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 12.7875
Epoch 10/10
1/1 [==============================] - 0s 498ms/step - loss: 19.5113
Test loss: 19.51128387451172
1/1 [==============================] - 0s 461ms/step


ValueError: ignored

In [ ]:
import numpy as np

# Define the neural network architecture
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize the weights with random binary values (0 or 1)
        self.input_to_hidden_weights = np.random.randint(2, size=(input_size, hidden_size))
        self.hidden_to_output_weights = np.random.randint(2, size=(hidden_size, output_size))

    def forward(self, x):
        # Forward pass through the network
        hidden_layer_input = x.dot(self.input_to_hidden_weights)
        hidden_layer_output = np.maximum(0, hidden_layer_input)  # ReLU activation
        output = hidden_layer_output.dot(self.hidden_to_output_weights)
        return output

# Create a random input vector
input_size=10
input_vector = np.random.randint(2, size=(1, input_size))

# Initialize the neural network
input_size = input_vector.shape[1]
hidden_size = 16  # Adjust the hidden layer size as needed
output_size = 1
neural_network = SimpleNeuralNetwork(input_size, hidden_size, output_size)

# Training loop through epochs
epochs = 10  # You can adjust the number of epochs as needed
for epoch in range(epochs):
    output = neural_network.forward(input_vector)

    # Ask the user if they are satisfied with the output and want to end the process
    user_satisfaction = input(f"Epoch {epoch + 1}: Output is {output[0, 0]}. Are you satisfied with this weight? (yes/no): ").strip().lower()

    if user_satisfaction == 'yes':
        print("Training ended. Weight is satisfactory.")
        break
    elif epoch == epochs - 1:
        print("Training completed. Maximum epochs reached.")
    else:
        # Update weights based on importance (you can define your own weight update logic here)
        # For this example, we won't update the weights, but you can add your custom logic.
        pass


Epoch 1: Output is 35. Are you satisfied with this weight? (yes/no): no
Epoch 2: Output is 35. Are you satisfied with this weight? (yes/no): yes
Training ended. Weight is satisfactory.


In [ ]:
import numpy as np

# Define the neural network architecture
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize the weights with random binary values (0 or 1)
        self.input_to_hidden_weights = np.random.randint(2, size=(input_size, hidden_size))
        self.hidden_to_output_weights = np.random.randint(2, size=(hidden_size, output_size))

    def forward(self, x):
        # Forward pass through the network
        hidden_layer_input = x.dot(self.input_to_hidden_weights)
        hidden_layer_output = np.maximum(0, hidden_layer_input)  # ReLU activation
        output = hidden_layer_output.dot(self.hidden_to_output_weights)
        return output

# Create a random input vector
input_vector = np.random.randint(2, size=(1, input_size))

# Initialize the weight vector W with random binary values (0 or 1)
W = np.random.randint(2, size=input_size)

# Initialize the neural network
input_size = input_vector.shape[1]
hidden_size = 16  # Adjust the hidden layer size as needed
output_size = 1
neural_network = SimpleNeuralNetwork(input_size, hidden_size, output_size)

# Training loop through epochs
epochs = 10  # You can adjust the number of epochs as needed
for epoch in range(epochs):
    output = neural_network.forward(input_vector)

    # Ask the user if they are satisfied with the weight vector W
    user_satisfaction = input(f"Epoch {epoch + 1}: Weight vector W is {W}. Are you satisfied with this weight? (yes/no): ").strip().lower()

    if user_satisfaction == 'yes':
        print("Training ended. Weight vector is satisfactory.")
        break
    elif epoch == epochs - 1:
        print("Training completed. Maximum epochs reached.")
    else:
        # Update each item in W based on importance (custom update logic)
        update_probability = 0.2  # Example: 20% chance to update each bit
        for i in range(len(W)):
            if np.random.rand() < update_probability:
                W[i] = 1 - W[i]  # Flip the bit (0 to 1 or 1 to 0)


Epoch 1: Weight vector W is [0 1 1 1 1 0 0 0 1 1]. Are you satisfied with this weight? (yes/no): no
Epoch 2: Weight vector W is [1 1 1 0 1 0 0 0 0 1]. Are you satisfied with this weight? (yes/no): no
Epoch 3: Weight vector W is [1 1 1 0 1 0 1 0 0 1]. Are you satisfied with this weight? (yes/no): yes
Training ended. Weight vector is satisfactory.


In [ ]:
import numpy as np

# Define the neural network architecture
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize the weights with random values in the range [0, 1]
        self.input_to_hidden_weights = np.random.rand(input_size, hidden_size)
        self.hidden_to_output_weights = np.random.rand(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through the network
        hidden_layer_input = x.dot(self.input_to_hidden_weights)
        hidden_layer_output = np.maximum(0, hidden_layer_input)  # ReLU activation
        output = hidden_layer_output.dot(self.hidden_to_output_weights)
        return output

# Create a random input vector
input_vector = np.random.rand(1, input_size)

# Initialize the weight vector W with all values set to 1
W = np.ones(input_size)

# Initialize the neural network
input_size = input_vector.shape[1]
hidden_size = 16  # Adjust the hidden layer size as needed
output_size = 1
neural_network = SimpleNeuralNetwork(input_size, hidden_size, output_size)

# Training loop through epochs
epochs = 10  # You can adjust the number of epochs as needed
for epoch in range(epochs):
    output = neural_network.forward(input_vector)

    # Ask the user if they are satisfied with the weight vector W
    user_satisfaction = input(f"Epoch {epoch + 1}: Weight vector W is {W}. Are you satisfied with this weight? (yes/no): ").strip().lower()

    if user_satisfaction == 'yes':
        print("Training ended. Weight vector is satisfactory.")
        break
    elif epoch == epochs - 1:
        print("Training completed. Maximum epochs reached.")
    else:
        # Update each item in W based on the importance (custom update logic)
        importance = np.random.rand(input_size)  # Example: Random importance values
        W = importance  # Set W to importance values (in the range [0, 1])


Epoch 1: Weight vector W is [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]. Are you satisfied with this weight? (yes/no): no
Epoch 2: Weight vector W is [0.53254777 0.98867734 0.93808052 0.32642869 0.87241635 0.77928975
 0.25031778 0.82932179 0.70338364 0.30112029]. Are you satisfied with this weight? (yes/no): no
Epoch 3: Weight vector W is [0.87660387 0.09825089 0.46763387 0.12227683 0.34124843 0.01208388
 0.61646199 0.79845924 0.5667413  0.2014135 ]. Are you satisfied with this weight? (yes/no): yes
Training ended. Weight vector is satisfactory.


In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds
ratings = tfds.load("movielens/100k-ratings")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteWRMJO3/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [ ]:
def convert_tf_to_pd(ds, limit=32):
    """
    Read data from Tensorflow dataset to Pandas dataframe
    :param ds:
    :param limit:
    :return:
    """
    batch_iterator = ds.batch(limit).make_one_shot_iterator()
    with tf.Session() as sess:
        batch = batch_iterator.get_next()
        features_and_labels = sess.run(batch)
        samples = {
            **features_and_labels[0],
            'label': features_and_labels[1],
        }

    return pd.DataFrame.from_dict(samples)

In [ ]:
ratings=convert_tf_to_pd(ratings)

NameError: ignored

In [ ]:
ratings.head()
movies.head()

AttributeError: ignored

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Sample user profiles and interaction data (replace with your real data)
user_profiles = np.random.rand(100, 20)  # 100 users with 20 features
interaction_data = np.random.randint(2, size=(100, 50))  # Binary interaction data

# Define the recommendation model with user profiles
class AttentionRecommendationModelWithProfiles(keras.Model):
    def __init__(self, num_users, num_items, num_features, **kwargs):
        super(AttentionRecommendationModelWithProfiles, self).__init__(**kwargs)
        self.user_embedding = keras.layers.Embedding(num_users, num_features)
        self.item_embedding = keras.layers.Embedding(num_items, num_features)
        self.attention_user = keras.layers.Attention()
        self.attention_item = keras.layers.Attention()
        self.dense = keras.layers.Dense(1)

    def call(self, inputs):
        user_profiles, item_ids = inputs
        user_embedded = self.user_embedding(item_ids)
        user_attention_weights = self.attention_user([user_embedded, user_profiles])
        weighted_user_embedding = tf.reduce_sum(user_attention_weights * user_embedded, axis=1)
        item_embedded = self.item_embedding(item_ids)
        item_attention_weights = self.attention_item([item_embedded, user_profiles])
        weighted_item_embedding = tf.reduce_sum(item_attention_weights * item_embedded, axis=1)
        return self.dense(weighted_user_embedding + weighted_item_embedding)

num_users=100
num_items=100
num_features=20
# Initialize and compile the model
model_with_profiles = AttentionRecommendationModelWithProfiles(num_users, num_items, num_features)
model_with_profiles.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on user profiles and interaction data
model_with_profiles.fit([user_profiles, item_ids_train], train_labels, epochs=10, batch_size=64)

# Make recommendations using user profiles and interactions
user_profile = user_profiles[0]  # Replace with the actual user's profile
user_profile = np.expand_dims(user_profile, axis=0)
recommended_items = model_with_profiles.predict([user_profile, item_ids_test])


NameError: ignored

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load and preprocess the MovieLens 100k dataset (user ratings, movie information, user profiles)

# Create user and movie embeddings using TensorFlow or other libraries
# Implement an attention mechanism

# Define the recommendation model with attention
class RecommendationModel(tf.keras.Model):
    def __init__(self):
        super(RecommendationModel, self).__init__()
        # Define layers and attention mechanism

    def call(self, user_embedding, movie_embedding, user_profile):
        # Implement the attention mechanism and combine embeddings
        # Return the recommendation scores

# Split the dataset into training and testing sets
# Train the recommendation model using user ratings and user embeddings

# Implement a simple conversational interface
while True:
    user_input = input("User: ")

    # Process user input to update user profile, e.g., gather preferences

    # Generate movie recommendations using the model
    user_embedding = # Get user embedding
    movie_embedding = # Get movie embedding
    user_profile = # Prepare user profile data
    recommendations = model(user_embedding, movie_embedding, user_profile)

    # Display recommended movies to the user
    print("Bot: Recommended Movies...")
    # Implement a method to choose the top-N movies based on model output

    user_feedback = input("User: Provide feedback or type 'continue': ")
    if user_feedback.lower() == 'continue':
        continue

    # Incorporate user feedback into the recommendation model for retraining

# Deploy the conversational recommendation system

# Continuously learn and adapt the model based on user feedback

# Evaluation and testing of the recommendation system

